<a href="https://colab.research.google.com/github/WyattVO/PytorchFineTuning/blob/main/PytorchTransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)

# 1. Data (CIFAR-10)
# ResNet18 was trained on ImageNet (3x224x224), so:
# - we need 224x224 images
# - we should normalize with ImageNet stats
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    transform=transform,
    download=True
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False)

# 2. Load pretrained model
# weights arg name in current torchvision is often weights=models.ResNet18_Weights.IMAGENET1K_V1
# but using the string you gave:
model = models.resnet18(weights='IMAGENET1K_V1')

# 3. Freeze all params
for param in model.parameters():
    param.requires_grad = False #makes so you cant edit the parameters.

# 4. Replace final layer (fc) with a new trainable head
# ResNet-18 ends with model.fc which is Linear(in_features, 1000)
# We want 10 classes for CIFAR-10.
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)#last 10 neural nodes

# Only the new layer's params require grad now
model = model.to(device)

# 5. Loss + optimizer (only optimize the last layer)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)
#using SGD model.
# 6. Training loop (just a couple epochs)
num_epochs = 5
val_acc_per_epoch = [] #Records accuracy.
for epoch in range(num_epochs):
    model.train()  # we're training the new head, since everything else is frozen.
    running_loss = 0.0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward (only updates model.fc)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] "
                  f"Batch {batch_idx} "
                  f"Loss: {loss.item():.4f}")

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] Avg Loss: {avg_loss:.4f}")

# 7. Eval on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)           # logits
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    val_acc_per_epoch.append(test_acc)
    print(f"[Epoch {epoch+1}/{num_epochs}] Val Acc: {test_acc:.2f}%")
print("Validation accuracy by epoch:", val_acc_per_epoch)
# Validation accuracy by epoch: [78.83, 77.78, 79.16, 77.41, 78.66]



device: cuda
Epoch [1/5] Batch 0 Loss: 2.4020
Epoch [1/5] Batch 100 Loss: 0.8487
Epoch [1/5] Batch 200 Loss: 0.6049
Epoch [1/5] Batch 300 Loss: 0.7748
Epoch [1/5] Batch 400 Loss: 0.6990
Epoch [1/5] Batch 500 Loss: 0.6419
Epoch [1/5] Batch 600 Loss: 0.6329
Epoch [1/5] Batch 700 Loss: 0.7202
Epoch [1/5] Avg Loss: 0.7314
[Epoch 1/5] Val Acc: 78.83%
Epoch [2/5] Batch 0 Loss: 0.7450
Epoch [2/5] Batch 100 Loss: 0.7412
Epoch [2/5] Batch 200 Loss: 0.6113
Epoch [2/5] Batch 300 Loss: 0.7277
Epoch [2/5] Batch 400 Loss: 0.7203
Epoch [2/5] Batch 500 Loss: 0.6646
Epoch [2/5] Batch 600 Loss: 0.5215
Epoch [2/5] Batch 700 Loss: 0.6449
Epoch [2/5] Avg Loss: 0.6411
[Epoch 2/5] Val Acc: 77.78%
Epoch [3/5] Batch 0 Loss: 0.6267
Epoch [3/5] Batch 100 Loss: 0.8739
Epoch [3/5] Batch 200 Loss: 0.6559
Epoch [3/5] Batch 300 Loss: 0.3322
Epoch [3/5] Batch 400 Loss: 0.6344
Epoch [3/5] Batch 500 Loss: 0.5718
Epoch [3/5] Batch 600 Loss: 0.6729
Epoch [3/5] Batch 700 Loss: 0.9115
Epoch [3/5] Avg Loss: 0.6256
[Epoch 3/5